In [2]:
import os
from datetime import datetime
import data_preparation as sf
import rnn_enc_dec as red


#PROJECT_ROOT_DIR = "/home/ubuntu/TroubledLife"
PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

In [3]:
#sf.generate_life_policy_data(no_of_policies=10000, runtime=5, trouble=False, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#sf.generate_life_policy_data(no_of_policies=2000, runtime=5, trouble=False, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    sf.load_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    sf.load_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)

In [4]:
# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     sf.pad_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     sf.pad_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
train_labels, train_features, train_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length)
test_labels, test_features, test_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length)

train_data = sf.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [5]:
import tensorflow as tf


tf.reset_default_graph()
tf.set_random_seed(42)

# Our hyperparameters
n_steps = max_policy_history_length
n_inputs = 2
n_layers = 3
n_neurons = 200
n_latent = max_policy_history_length * 2
learning_rate = 0.0001
bidirectional = True

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None], name="seq_length")
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")

latent_vector = red.encoder(X=X, seq_length=seq_length, n_layers=n_layers, n_neurons=n_neurons, n_latent=n_latent)

decoder_sequence_test = red.decoder(n_latent=n_latent, n_layers=n_layers, n_neurons=n_neurons, n_outputs=n_inputs, 
                               latent_vector=latent_vector, X=X, seq_length=seq_length, training=True, reuse=False)

tvars = tf.trainable_variables()

for var in tvars:
    print(var) 
    
loss_test = tf.reduce_mean(tf.square(decoder_sequence_test - X)) # MSE

# Both are needed when using precision and recall metrics
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

saver = tf.train.Saver(var_list=tvars)

encoder states shape: (?, 600)
latent_vector shape: (?, 30)
state_input shape: (?, 200)
decoder initial state shape:  (?, 600)


decoder outputs rnn_output shape: (?, ?, 2)
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(202, 200) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_1/basic_rnn_cell/kernel:0' shape=(400, 200) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_1/basic_rnn_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_2/basic_rnn_cell/kernel:0' shape=(400, 200) dtype=float32_ref>
<tf.Variable 'P_Encoder/rnn/multi_rnn_cell/cell_2/basic_rnn_cell/bias:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Encoder_2_Latent/W:0' shape=(600, 30) dtype=float32_ref>
<tf.Variable 'P_Encoder_2_Latent/b:0' shape=(30,) dtype=float32_ref>
<tf.Variable 'Latent_2_P_Decoder/W:0' shape=(30, 200) dtype=float32_ref>
<tf.Variable 'Latent_2_P_Decoder/b:0' shape=(200,) dtype=float32_ref>
<tf.Variable 'P_Decoder

In [6]:
from time import time

start = time()

n_epochs = 100
batch_size = 200

with tf.Session() as sess:
    init_g.run()
    init_l.run()

    saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "rnn_enc_dec.ckpt"))    
    
    lTest = loss_test.eval(feed_dict={X: test_features, seq_length: test_seq_lengths})

    print("Test loss:", lTest)
 
writer.close()

elapsed = time() - start
print("Time to finish %d", elapsed)



INFO:tensorflow:Restoring parameters from /Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/rnn_enc_dec.ckpt


InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on /Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/rnn_enc_dec.ckpt: Not found: /Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]

Caused by op 'save/RestoreV2_2', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/gopora/MyStuff/Dev/python-virtual-envs/pie36-tf13/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/gopora/MyStuff/Dev/py

In [6]:
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")


In [7]:
saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "rnn_enc_dec.ckpt"))    


RuntimeError: Attempted to use a closed Session.